In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense,\
                                      add, BatchNormalization, Activation

In [2]:
from subpixel_conv2d import SubpixelConv2D

In [3]:
from tensorflow.keras.models import Model

In [4]:
from model import get_D, get_G
#from subpixel_conv2d import SubpixelConv2D

In [5]:
import os
import time
import random
import numpy as np
from PIL import Image

import tensorflow as tf

from model import get_G, get_D
from utils import *


In [6]:
batch_size = 10  # use 4 if your GPU memory is small
lr_v = 1e-3
tot_sample= 100  # Totall traning images
## adversarial learning (SRGAN)
n_epoch = 400
## initialize G
n_epoch_init = n_epoch//5

# create folders to save result images and trained models
save_dir = "samples"
checkpoint_dir = "models"
save_ind= 0

if not os.path.exists(save_dir):
	os.makedirs(save_dir)
if not os.path.exists(checkpoint_dir):
	os.makedirs(checkpoint_dir)

In [7]:
# load Data
HR_train, LR_train= get_data('Data')

In [8]:
HR_train.shape, LR_train.shape

((100, 384, 384, 3), (100, 96, 96, 3))

In [9]:
np.savez('IRD.npz', name1=HR_train, name2=LR_train)

In [8]:
# load modols
G = get_G((96, 96, 3))
D = get_D((384, 384, 3))
vgg= get_vgg19()

In [9]:
vgg.predict(np.random.randn(1, 384, 384, 3))

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node functional_1/block1_conv1/Relu (defined at <ipython-input-9-93a20e733ef5>:1) ]] [Op:__inference_predict_function_2410]

Function call stack:
predict_function


In [15]:
# Optimizers
g_optimizer_init = tf.optimizers.Adam(lr_v)
g_optimizer = tf.optimizers.Adam(lr_v)
d_optimizer = tf.optimizers.Adam(lr_v)


## initialize learning (G)
n_step_epoch = round(n_epoch_init // batch_size)
for epoch in range(n_epoch_init):
    i,j= ((epoch)*batch_size)%tot_sample, (((epoch+1))*batch_size)%tot_sample
    X, Y= LR_train[i: j], HR_train[i: j]
    with tf.GradientTape() as tape:
        ypred= G(X)
        mse_loss= tf.reduce_mean(tf.reduce_mean(tf.math.squared_difference(Y, ypred), axis=-1))
        grad = tape.gradient(mse_loss, G.trainable_weights)
        g_optimizer_init.apply_gradients(zip(grad, G.trainable_weights))

    print("Epoch: [{}/{}] step: mse: {:.3f} ".format(
            epoch, n_epoch_init , mse_loss))
    if epoch%10 ==0:
        img= np.array(G.predict(LR_train[np.newaxis, save_ind]))
        img= Image.fromarray(img)
        img.save(os.path.join(save_dir, 'train_g_{}.png'.format(epoch)))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [16]:
G.predict(X)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node generator/conv2d/Relu (defined at <ipython-input-16-30b673bd21ac>:1) ]] [Op:__inference_predict_function_2619]

Function call stack:
predict_function


In [17]:
X.shape

(10, 96, 96, 3)